# 🪖 SRIG — Helmet Detection using RT-DETR
### Smart Road India Grid | Model 1 — Vision Transformer
**Yash Jani | Red and White Institute, Surat**

---

## 🧠 RT-DETR Kya Hai?

**RT-DETR = Real-Time Detection Transformer**

```
CCTV Image Input
       ↓
Backbone (ResNet)
Image ke features extract karta hai
       ↓
Transformer Encoder
Global context samajhta hai
(YOLOv8 se better — overlapping objects bhi detect karta hai)
       ↓
Transformer Decoder
Exact location predict karta hai
       ↓
Output: Bounding Box + Label
'with helmet' / 'without helmet'
```

### 🔑 YOLOv8 vs RT-DETR
| | YOLOv8 (CNN) | RT-DETR (Transformer) |
|---|---|---|
| Architecture | CNN based | Transformer based |
| Crowded scenes | Average | **Better** |
| Accuracy | 94.7% | **95%+** |
| SRIG fit | Good | **Best** |

## 📦 Step 1 — Libraries Import Karo

In [ ]:
import os
import yaml
import torch
from ultralytics import RTDETR
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✅ Libraries import ho gayi!')
print(f'💻 Device: {device}')
print(f'🔥 PyTorch: {torch.__version__}')

## 📂 Step 2 — Dataset Path Fix Karo

In [ ]:
# Absolute paths set karo — Windows pe zaroori hai
BASE_DIR  = Path('data').resolve()
TRAIN_DIR = BASE_DIR / 'train' / 'images'
VALID_DIR = BASE_DIR / 'valid' / 'images'
TEST_DIR  = BASE_DIR / 'test'  / 'images'

print(f'📁 Base  : {BASE_DIR}')
print(f'📁 Train : {TRAIN_DIR}')
print(f'📁 Valid : {VALID_DIR}')
print(f'📁 Test  : {TEST_DIR}')

# Count images
train_count = len(list(TRAIN_DIR.glob('*.jpg')) + list(TRAIN_DIR.glob('*.png')))
valid_count = len(list(VALID_DIR.glob('*.jpg')) + list(VALID_DIR.glob('*.png')))
test_count  = len(list(TEST_DIR.glob('*.jpg'))  + list(TEST_DIR.glob('*.png')))

print(f'\n📊 Dataset Summary:')
print(f'   Train images : {train_count}')
print(f'   Valid images : {valid_count}')
print(f'   Test images  : {test_count}')
print(f'   Total        : {train_count + valid_count + test_count}')

## ⚙️ Step 3 — data.yaml Update Karo

RT-DETR ko absolute paths chahiye — relative paths se error aata hai.

In [ ]:
# Updated yaml content
yaml_content = {
    'train': str(TRAIN_DIR),
    'val':   str(VALID_DIR),
    'test':  str(TEST_DIR),
    'nc':    2,
    'names': ['with helmet', 'without helmet']
}

yaml_path = BASE_DIR / 'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

print(f'✅ data.yaml updated!')
print(f'   Path: {yaml_path}')

# Verify
with open(yaml_path) as f:
    print(f'\n📄 Content:')
    print(f.read())

## 🖼️ Step 4 — Sample Images Dekho

In [ ]:
# Kuch sample images dikhao
sample_images = list(TRAIN_DIR.glob('*.jpg'))[:6]
if not sample_images:
    sample_images = list(TRAIN_DIR.glob('*.png'))[:6]

if sample_images:
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    fig.suptitle('SRIG — Sample Training Images', fontsize=14, fontweight='bold')
    axes = axes.flatten()
    for i, img_path in enumerate(sample_images):
        img = mpimg.imread(str(img_path))
        axes[i].imshow(img)
        axes[i].set_title(img_path.name[:20], fontsize=8)
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()
    print(f'✅ {len(sample_images)} sample images dikhayi!')
else:
    print('⚠️ Images nahi mili — path check karo!')

## 🤖 Step 5 — RT-DETR Model Load Karo

Pretrained model download hoga — **~160MB** — thoda wait karo.

In [ ]:
# RT-DETR small model load karo (CPU pe bhi chalega)
model = RTDETR('rtdetr-l.pt')

print('✅ RT-DETR model load ho gaya!')
print('   Model: RT-DETR Large')
print('   Pre-trained on: COCO Dataset (80 classes)')
print('   Ab hum ise helmet detection ke liye fine-tune karenge!')

## 🏋️ Step 6 — Model Train Karo

⚠️ **CPU pe 1-2 ghante lag sakte hain** — patience rakho!

Training ke dauran yeh dikhega:
```
Epoch 1/20:  box_loss, cls_loss, mAP50
Epoch 2/20:  ...
```

In [ ]:
results = model.train(
    data=str(yaml_path),   # dataset path
    epochs=20,             # 20 rounds of training
    imgsz=640,             # image size
    batch=4,               # CPU ke liye small batch
    device=device,         # cpu ya cuda
    project='models',      # save location
    name='helmet_rtdetr',  # folder name
    patience=5,            # early stopping
    save=True,
    plots=True,            # training graphs save karo
    verbose=True
)

print('\n🎉 Training complete!')
print(f'   Model saved: models/helmet_rtdetr/')

## 📊 Step 7 — Training Results Dekho

In [ ]:
results_path = Path('models/helmet_rtdetr')

# Results image dikhao
result_img = results_path / 'results.png'
if result_img.exists():
    img = mpimg.imread(str(result_img))
    plt.figure(figsize=(16, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('RT-DETR Training Results', fontsize=14, fontweight='bold')
    plt.show()
else:
    print('Results image generate ho rahi hai...')

# mAP score print karo
print(f'\n🎯 Final Results:')
print(f'   mAP50  : {results.results_dict.get("metrics/mAP50(B)", 0):.3f}')
print(f'   mAP50-95: {results.results_dict.get("metrics/mAP50-95(B)", 0):.3f}')

## 🔍 Step 8 — Test Image pe Predict Karo

In [ ]:
# Best trained model load karo
best_model = RTDETR('models/helmet_rtdetr/weights/best.pt')

# Test image pe predict karo
test_images = list(TEST_DIR.glob('*.jpg'))[:3]
if not test_images:
    test_images = list(TEST_DIR.glob('*.png'))[:3]

if test_images:
    predictions = best_model.predict(
        source=str(test_images[0]),
        conf=0.5,
        save=True,
        project='models',
        name='helmet_predictions'
    )

    # Result dikhao
    pred_dir = Path('models/helmet_predictions')
    pred_imgs = list(pred_dir.glob('*.jpg'))
    if pred_imgs:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle('SRIG — Helmet Detection Result', fontsize=14, fontweight='bold')

        # Original
        orig = mpimg.imread(str(test_images[0]))
        axes[0].imshow(orig)
        axes[0].set_title('Original Image')
        axes[0].axis('off')

        # Prediction
        pred = mpimg.imread(str(pred_imgs[0]))
        axes[1].imshow(pred)
        axes[1].set_title('RT-DETR Prediction')
        axes[1].axis('off')

        plt.tight_layout()
        plt.savefig('models/detection_result.png', dpi=150, bbox_inches='tight')
        plt.show()
        print('✅ Detection result save ho gaya: models/detection_result.png')

    # Detection details
    for r in predictions:
        print(f'\n📋 Detections:')
        for box in r.boxes:
            cls  = int(box.cls)
            conf = float(box.conf)
            name = ['with helmet', 'without helmet'][cls]
            emoji = '✅' if cls == 0 else '❌'
            print(f'   {emoji} {name} — Confidence: {conf:.1%}')
else:
    print('⚠️ Test images nahi mili!')

## 🎉 Step 9 — SRIG Integration Summary

In [ ]:
print('='*55)
print('       🚦 SRIG — MODEL STATUS DASHBOARD')
print('='*55)
print('  ✅ Model 3 — Civic Score (TabTransformer)  DONE')
print('  ✅ Model 1 — Helmet Detection (RT-DETR)    DONE')
print('  ⏳ Model 2 — School Zone Speed             NEXT')
print('  ⏳ Model 4 — Accident Hotspot              NEXT')
print('='*55)
print('\n🔗 SRIG Pipeline:')
print('  CCTV Frame')
print('      ↓')
print('  RT-DETR → Helmet Detected? → Score Update')
print('      ↓')
print('  TabTransformer → Risk Level → Alert/Reward')
print('='*55)